<a href="https://colab.research.google.com/github/dsivov/MyColabs/blob/main/summ_experiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Install dependences - may take a while  
! pip install sumy
! pip install --index-url http://74.82.29.209:8090 nebula3_database==0.2.2 --trusted-host 74.82.29.209
! pip install spacy
! pip install openai
! pip install fastcoref
! pip install ipywidgets
from IPython.display import clear_output
clear_output()

In [ ]:
#@title Imports 
import nltk
nltk.download('punkt')
from ipywidgets import interact
import ipywidgets as widgets
from IPython.display import display
from database.arangodb import ArangoClient 
import openai

In [ ]:
#@title Get data from Arango
dbname = "web_demo"
arango_host = "http://172.83.9.249:8529"
client = ArangoClient(hosts=arango_host)
db = client.db(dbname, username='nebula', password='nebula')
movies = []
urls = {}
for movie in db.collection("Movies").all():
    movies.append(movie['_id'])
    urls[movie['_id']] = movie['url_path']
movies.append('Free_Text')
movies.reverse()


In [ ]:
#@title Widgets
movie_id = widgets.Dropdown(
    options=movies,
    value='Free_Text',
    description='Movie ID',
    disabled=False,
    style={'description_width': 'initial'},
    layout = widgets.Layout(width='80%')
)
api_key = widgets.Text(
        value='12345678',
        placeholder='Type something',
        description='API KEY',
        disabled=False,
        style={'description_width': 'initial'},
        layout = widgets.Layout(width='80%')
    )
free_text = widgets.Textarea(
        value='',
        placeholder='Type something',
        description='Free text',
        disabled=False,
        style={'description_width': 'initial'},
        layout = widgets.Layout(height="auto", width="80%")
    )
prompt = widgets.Text(
        value='Abstractive summarization of following sentences:',
        placeholder='Type something',
        description='Prompt ',
        disabled=False,
        style={'description_width': 'initial'},
        layout = widgets.Layout(height="auto", width="80%")
    )


In [ ]:
#@title Input API Key and Movie ID
display(api_key)
print()
display(movie_id)

In [ ]:
movie_server = "http://74.82.29.209:9000/"
movie_url = movie_server + urls[movie_id.value]
print(movie_url)

In [ ]:
#@title Get captions 
if movie_id.value == 'Free_Text':
    display(free_text)
    print()
else:
    mdfs = []
    cbs = []
    print(movie_id.value)
    for res in db.collection("s4_llm_output").find({'movie_id': movie_id.value}):
        mdfs.append({'frame_num':res['frame_num'], 'captions': res['candidate']})
        temp =widgets.Checkbox(
        value=True,
        description=res['candidate'],
        disabled=False,
        indent=False,
        layout = widgets.Layout(width='100%')
        )
        display(temp)
        cbs.append(temp)

#print(mdfs)

In [ ]:
#@title Seleted captions 
if movie_id.value == 'Free_Text':
    text = free_text.value
else:
    text = ""
    for cb in cbs:
        if cb.value == True:
            text = text + cb.description
print(text)


In [ ]:
#@title Run simple summization by SUMY
from __future__ import absolute_import
from __future__ import division, print_function, unicode_literals

from sumy.parsers.html import HtmlParser
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lsa import LsaSummarizer as Summarizer
from sumy.nlp.stemmers import Stemmer
from sumy.utils import get_stop_words


LANGUAGE = "english"
SENTENCES_COUNT = 10


#prompt = "Summarise following text:\n"
#prompt = "Extract 5 important actions from following text:\n"
sum = ""
#prompt = "Generate long title from captions:\n"
#prompt = "Generate few captivating headings from captions:\n"
#prompt = "Create few summarising headings from captions:\n"
#prompt = "what happens in the scene described by the following text?:\n"
parser = PlaintextParser.from_string(text, Tokenizer(LANGUAGE))    
stemmer = Stemmer(LANGUAGE)

summarizer = Summarizer(stemmer)
summarizer.stop_words = get_stop_words(LANGUAGE)

for sentence in summarizer(parser.document, SENTENCES_COUNT):
    sum = sum + str(sentence)
print(sum)

In [ ]:
#@title Co-ref 
from fastcoref import spacy_component
import spacy
#!python3 -m spacy download en_core_web_sm


#text = 'Alice goes down the rabbit hole. Where she would discover a new reality beyond her expectations.'

nlp = spacy.load("en_core_web_sm")
nlp.add_pipe("fastcoref")

doc = nlp(sum, component_cfg={"fastcoref": {'resolve_text': True}})
print(doc._.coref_clusters)
print(doc._.resolved_text)
sum = doc._.resolved_text

In [ ]:
#@title Input prompt
display(prompt)

In [ ]:
#@title Print final text to sum
print(prompt.value + "\n" + sum)
_to_gpt = prompt.value + "\n" + sum

In [ ]:
#@title GPT-3 API
openai.api_key = api_key.value
#prompt = "Generate captivating heading from captions of movie:\n" + text
#print(prompt)
response = openai.Completion.create(
  engine="text-davinci-003",
  prompt=_to_gpt,
  temperature=0.0,
  max_tokens=256
  #top_p=1.0,
  #frequency_penalty=0.0,
  #presence_penalty=0.0
)


print(response['choices'][0]['text'])
